<a href="https://colab.research.google.com/github/zzhining/stock_market_analysis/blob/main/3%EC%A3%BC%EC%B0%A8/%EC%B0%A8%ED%8A%B8%EA%B8%B0%EB%B0%98_%EC%A3%BC%EA%B0%80%EB%B6%84%EC%84%9D_%EC%BA%94%EB%93%A4%ED%8C%A8%ED%84%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 데이터 준비 및 환경 설정

In [ ]:
import os
os.chdir("../../데이터")

In [ ]:
import pandas as pd
sp_data_dict = dict()
for file_name in os.listdir("주가데이터"):
    sp_data = pd.read_csv("주가데이터/" + file_name, parse_dates = ['Date'])
    stock_name = file_name.replace('.csv', '')
    sp_data_dict[stock_name] = sp_data

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import rcParams
sns.set()
%matplotlib inline
rcParams['font.family'] = 'Malgun Gothic'
rcParams['axes.unicode_minus'] = False

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### 상승장악형

#### 패턴 탐지

In [ ]:
# 현대차 예시
sample_data = sp_data_dict["현대차"]

In [ ]:
import numpy as np
cond_1 = (sample_data['Open'] > sample_data['Close']).values[:-1]
cond_2 = (sample_data['Open'] < sample_data['Close']).values[1:]
cond_3 = (sample_data['Low'].values[:-1] > sample_data['Open'].values[1:])
cond_4 = (sample_data['High'].values[:-1] < sample_data['Close'].values[1:])

cond = cond_1 & cond_2 & cond_3 & cond_4
cond = np.insert(cond, 0, False)
sample_data["상승장악형"] = cond

In [ ]:
# 전종목 적용
for stock_name in sp_data_dict.keys():
    sp_data = sp_data_dict[stock_name]
    cond_1 = (sp_data['Open'] > sp_data['Close']).values[:-1]
    cond_2 = (sp_data['Open'] < sp_data['Close']).values[1:]
    cond_3 = (sp_data['Low'].values[:-1] > sp_data['Open'].values[1:])
    cond_4 = (sp_data['High'].values[:-1] < sp_data['Close'].values[1:])

    cond = cond_1 & cond_2 & cond_3 & cond_4
    cond = np.insert(cond, 0, False)
    sp_data["상승장악형"] = cond

#### 기대 수익 계산

In [ ]:
def calc_ror_after_patt(sp_data, patt_col, n):
    patt_idx_list = sp_data.loc[sp_data[patt_col]].index
    no_patt_idx_list = sp_data.index.difference(patt_idx_list)
    max_idx = max(sp_data.index)

    patt_bidx_list = patt_idx_list[patt_idx_list + n < max_idx] + 1
    patt_sidx_list = patt_bidx_list + n
    no_patt_bidx_list = no_patt_idx_list[no_patt_idx_list + n < max_idx] + 1
    no_patt_sidx_list = no_patt_bidx_list + n
    # 패턴 발생 후 수익률 계산
    patt_bp_list = sp_data.loc[patt_bidx_list, 'Close'].values
    patt_sp_list = sp_data.loc[patt_sidx_list, 'Close'].values
    patt_ror_list = (patt_sp_list - patt_bp_list) / patt_bp_list * 100
    # 패턴 미 발생 후 수익률 계산
    no_patt_bp_list = sp_data.loc[no_patt_bidx_list, 'Close'].values
    no_patt_sp_list = sp_data.loc[no_patt_sidx_list, 'Close'].values
    no_patt_ror_list = (no_patt_sp_list - no_patt_bp_list) / no_patt_bp_list * 100

    return patt_ror_list.tolist(), no_patt_ror_list.tolist()

In [ ]:
result_dict = dict()
for n in [5, 20, 60, 120]:
    total_patt_ror_list = []
    total_no_patt_ror_list = []
    for stock_name in sp_data_dict.keys():
        sp_data = sp_data_dict[stock_name]
        patt_ror_list, no_patt_ror_list = calc_ror_after_patt(sp_data, "상승장악형", n)
        total_patt_ror_list += patt_ror_list
        total_no_patt_ror_list += no_patt_ror_list
    result_dict[n] = total_patt_ror_list, total_no_patt_ror_list

#### 5영업일 보유 시 수익 비교

In [ ]:
plt.title("상승장악형 - 5영업일 이후")
plt.boxplot(result_dict[5])
plt.xticks([1, 2], ["패턴발생", "패턴미발생"])
plt.ylim(-100, 100)
plt.show()

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[5]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 20영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[20]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 60영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[60]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 120영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[120]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

### 하락장악형

#### 패턴 탐지

In [ ]:
for stock_name in sp_data_dict.keys():
    sp_data = sp_data_dict[stock_name]
    cond_1 = (sp_data['Open'] < sp_data['Close']).values[:-1]
    cond_2 = (sp_data['Open'] > sp_data['Close']).values[1:]
    cond_3 = (sp_data['High'].values[:-1] < sp_data['Open'].values[1:])
    cond_4 = (sp_data['Low'].values[:-1] > sp_data['Close'].values[1:])

    cond = cond_1 & cond_2 & cond_3 & cond_4
    cond = np.insert(cond, 0, False)
    sp_data["하락장악형"] = cond

#### 기대 수익 계산

In [ ]:
result_dict = dict()
for n in [5, 20, 60, 120]:
    total_patt_ror_list = []
    total_no_patt_ror_list = []
    for stock_name in sp_data_dict.keys():
        sp_data = sp_data_dict[stock_name]
        patt_ror_list, no_patt_ror_list = calc_ror_after_patt(sp_data, "하락장악형", n)
        total_patt_ror_list += patt_ror_list
        total_no_patt_ror_list += no_patt_ror_list
    result_dict[n] = total_patt_ror_list, total_no_patt_ror_list

#### 5영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[5]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 20영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[20]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 60영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[60]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 120영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[120]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

### 적삼병

#### 패턴 탐지

In [ ]:
for stock_name in sp_data_dict.keys():
    sp_data = sp_data_dict[stock_name]
    cur_price = sp_data["Close"].values[2:]
    pre_price = sp_data["Close"].values[1:-1]
    sec_pre_price = sp_data["Close"].values[:-2]

    cond_1 = (cur_price > pre_price) & (pre_price > sec_pre_price)
    cond_2 = (sp_data['Open'] < sp_data['Close']).values[:-2]
    cond_3 = (sp_data['Open'] < sp_data['Close']).values[1:-1]
    cond_4 = (sp_data['Open'] < sp_data['Close']).values[2:]

    cond = cond_1 & cond_2 & cond_3 & cond_4
    cond = np.insert(cond, [0, 0], False)
    sp_data["적삼병"] = cond

#### 기대 수익 계산

In [ ]:
result_dict = dict()
for n in [5, 20, 60, 120]:
    total_patt_ror_list = []
    total_no_patt_ror_list = []
    for stock_name in sp_data_dict.keys():
        sp_data = sp_data_dict[stock_name]
        patt_ror_list, no_patt_ror_list = calc_ror_after_patt(sp_data, "적삼병", n)
        total_patt_ror_list += patt_ror_list
        total_no_patt_ror_list += no_patt_ror_list
    result_dict[n] = total_patt_ror_list, total_no_patt_ror_list

#### 5영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[5]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 20영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[20]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 60영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[60]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 120영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[120]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

### 흑삼병

#### 패턴 탐지

In [ ]:
for stock_name in sp_data_dict.keys():
    sp_data = sp_data_dict[stock_name]
    cur_price = sp_data["Close"].values[2:]
    pre_price = sp_data["Close"].values[1:-1]
    sec_pre_price = sp_data["Close"].values[:-2]

    cond_1 = (cur_price < pre_price) & (pre_price < sec_pre_price)
    cond_2 = (sp_data['Open'] > sp_data['Close']).values[:-2]
    cond_3 = (sp_data['Open'] > sp_data['Close']).values[1:-1]
    cond_4 = (sp_data['Open'] > sp_data['Close']).values[2:]

    cond = cond_1 & cond_2 & cond_3 & cond_4
    cond = np.insert(cond, [0, 0], False)
    sp_data["흑삼병"] = cond

#### 기대 수익 계산

In [ ]:
result_dict = dict()
for n in [5, 20, 60, 120]:
    total_patt_ror_list = []
    total_no_patt_ror_list = []
    for stock_name in sp_data_dict.keys():
        sp_data = sp_data_dict[stock_name]
        patt_ror_list, no_patt_ror_list = calc_ror_after_patt(sp_data, "흑삼병", n)
        total_patt_ror_list += patt_ror_list
        total_no_patt_ror_list += no_patt_ror_list
    result_dict[n] = total_patt_ror_list, total_no_patt_ror_list

#### 5영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[5]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 20영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[20]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 60영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[60]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 120영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[120]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

### 샛별형

#### 패턴 탐지

In [ ]:
for stock_name in sp_data_dict.keys():
    sp_data = sp_data_dict[stock_name]
    cond_1 = sp_data["Close"].values[:-2] <= sp_data["Open"].values[:-2] * 1.1
    cond_2 = sp_data["Close"].values[1:-1] > sp_data["Open"].values[1:-1]
    cond_3 = sp_data["Close"].values[1:-1] < sp_data["Open"].values[1:-1] * 1.01
    cond_4 = sp_data["Close"].values[2:] >= sp_data["Open"].values[2:] * 1.1

    cond = cond_1 & cond_2 & cond_3 & cond_4
    cond = np.insert(cond, [0, 0], False)
    sp_data["샛별형"] = cond

#### 기대 수익 계산

In [ ]:
result_dict = dict()
for n in [5, 20, 60, 120]:
    total_patt_ror_list = []
    total_no_patt_ror_list = []
    for stock_name in sp_data_dict.keys():
        sp_data = sp_data_dict[stock_name]
        patt_ror_list, no_patt_ror_list = calc_ror_after_patt(sp_data, "샛별형", n)
        total_patt_ror_list += patt_ror_list
        total_no_patt_ror_list += no_patt_ror_list
    result_dict[n] = total_patt_ror_list, total_no_patt_ror_list

#### 5영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[5]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 20영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[20]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 60영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[60]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))

#### 120영업일 보유 시 수익 비교

In [ ]:
total_patt_ror_list, total_no_patt_ror_list = result_dict[120]
result = pd.concat([pd.Series(total_patt_ror_list).describe(),
                   pd.Series(total_no_patt_ror_list).describe()],
                   axis = 1)
result.columns = ['패턴발생', '패턴미발생']
display(result.round(2))